# **Rewrite-Retrieve-Read (RRR)**
このフレームワークは、入力クエリをLLＭを使い改良することで正確な出力を生成することに重点を置いています。

Research Paper: [Rewrite-Retrieve-Read](https://arxiv.org/pdf/2305.14283)

In [97]:
# set apikey to environment
import os
from dotenv import load_dotenv
load_dotenv()
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["LANGSMITH_API_KEY"] = os.getenv("LANGSMITH_API_KEY") # 使ってみる

In [98]:
# load japanese data from PDF
from langchain_community.document_loaders import PyPDFLoader
file_path = "../data/pdf/57_public_スタートアップ育成に向けた政府の取組_file_name=kaisetsushiryou_2024.pdf"
loader = PyPDFLoader(file_path)
documents = loader.load_and_split()

# load embedding model
from langchain_openai import OpenAIEmbeddings
embeddings = OpenAIEmbeddings()

# split pages content
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=0)
documents = text_splitter.split_documents(documents)

# load vectorstore
from langchain.vectorstores import Chroma # documentsをembeddingsでベクトル化し類似度で順位(index)をつけることで高速検索可能なベクトルデータベースを作ります。クエリとして入力された文章もベクトル化し、Chroma内に保存されたベクトルと比較して、類似度が高いものをランキング形式で返してくれます。
vectorstore = Chroma.from_documents(documents, embeddings)

# create retriever
retriever = vectorstore.as_retriever() # あとで baseのretrieverとして使う

# load llm
from langchain_openai import ChatOpenAI
llm = ChatOpenAI() # gpt-turbo-3
# create document chain
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

template = """"
You are a helpful assistant that answers questions based on the following context.
If you don't find the answer in the context, just say that "I don't know".

Context: {context}

Question: {input}

Answer:

"""
prompt = ChatPromptTemplate.from_template(template)


rag_chain = (
    {"context": retriever,  "input": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [99]:
# define simple query
simple_query = "スタートアップの経済効果は？"
# define distracted query
distracted_query = "スタートアップはもうかるの？"

# response
simple_query_response = rag_chain.invoke(simple_query)
print(f"簡潔な質問: {simple_query} 答え: {simple_query_response}")

# response
distracted_query_response = rag_chain.invoke(distracted_query)
print(f"ふわっとした質問: {distracted_query} 答え: {distracted_query_response}")

簡潔な質問: スタートアップの経済効果は？ 答え: スタートアップの経済効果は、直接効果で10.47兆円、間接波及効果を含めると19.39兆円と試算されています。
ふわっとした質問: スタートアップはもうかるの？ 答え: I don't know.


In [100]:
# define rewrite prompt for distracted query
template = """Provide a better search query for \
web search engine to answer the given question, end \
the queries with ’**’. Question: \
{x} Answer:""" # 改善されたクエルの最後には**がつく

rewrite_prompt = ChatPromptTemplate.from_template(template)


# parse response
def _parse(text):
    return text.strip('"').strip("**")

# creae rewiter chain
rewriter = rewrite_prompt | ChatOpenAI(temperature=0) | StrOutputParser() | _parse


In [ ]:
from langchain.callbacks import LangChainTracer
from langchain.callbacks.manager import CallbackManager

# LangChainTracerの初期化
tracer = LangChainTracer(project_name="rewrite_retrieve_read")
callback_manager = CallbackManager([tracer])# トレースを開始する際に_nameを指定

rewrite_retrieve_read_chain = (
    {
        "context": {"x": RunnablePassthrough()} | rewriter | retriever,
        "input": RunnablePassthrough(),
    }
    | prompt
    | llm
    | StrOutputParser()
)

# チェーンの実行
response = rewrite_retrieve_read_chain.invoke(distracted_query, config={"callbacks": callback_manager})
print(response)

スタートアップは急成長を目指す企業であり、経済成長のドライバーとされています。将来の所得や財政を支える新たな担い手となる可能性がありますが、利益を上げるかどうかは個々の事業や市場の状況により異なります。一概に「もうかる」と言えるわけではありません。


In [102]:
# LLMの出力を確認
llm_output = ChatOpenAI(temperature=0).invoke("スタートアップはもうかるの？")
print(f"LLMの出力: {llm_output}")

# _parse関数に渡す前の文字列
print(f"_parse関数に渡す前の文字列: {llm_output.content}")

# _parse関数を適用
parsed_output = _parse(llm_output.content)
print(f"_parse関数を適用した後の文字列: {parsed_output}")

LLMの出力: content='スタートアップが利益を上げるかどうかは、多くの要因に依存します。成功するスタートアップは、市場の需要に合った製品やサービスを提供し、効果的なマーケティング戦略を展開し、適切な資金調達を行うことが重要です。また、競合他社との差別化や顧客満足度の向上も成功につながる要因です。\n\nただし、スタートアップはリスクが高いビジネスであり、成功するかどうかは保証されません。多くのスタートアップは失敗し、利益を上げることが難しいとされています。しかし、適切な戦略やリーダーシップ、チームワークがあれば、成功する可能性もあります。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 257, 'prompt_tokens': 20, 'total_tokens': 277, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None} id='run-50070e3c-387b-4eaa-a3fc-2211ae2b73ca-0' usage_metadata={'input_tokens': 20, 'output_tokens': 257, 'total_tokens': 277, 'input_token_details': {'audio': 0, 'cache_read': 0}, 'output_token_details': {'audio': 0, 'reasoning': 0}}
_parse関数に渡す前の文字列: スタートアップが利益を上げるかど

# Evaluation with gpt4o-mini

In [103]:
# prepare data for evaluation
def evaluate_rag_chain(rag_chain, retriever, file_path):
    """
    RAGチェーンの評価を行い、評価結果のDataFrameを返します。
    
    Args:
        rag_chain: 評価対象のRAGチェーン
        retriever: 使用するretriever
        file_path: 評価対象のファイルパス
    
    Returns:
        pd.DataFrame: 評価結果を含むDataFrame
    """
    # データセットの読み込みとフィルタリング
    from datasets import load_dataset
    import pandas as pd
    
    test_data = load_dataset("allganize/RAG-Evaluation-Dataset-JA")["test"]
    
    # 評価対象ファイル名の抽出
    tgt_name = file_path.split("file_name=")[-1]
    
    # 評価対象データのフィルタリング
    filtered_data = test_data.filter(lambda x: x["target_file_name"] == tgt_name)
    eval_df = filtered_data.to_pandas()[["question", "target_answer"]]
    
    # RAGチェーンを使用して回答と文脈を取得
    p_data = []
    for _, row in eval_df.iterrows():
        q = row["question"]
        t = row["target_answer"]
        
        # RAGチェーンを実行
        response_text = rag_chain.invoke(q, config={"callbacks": callback_manager})
        context_docs = retriever.invoke(q, config={"callbacks": callback_manager})
        
        p_data.append({
            "question": q,
            "target_answer": t,
            "response": response_text,
            "context": context_docs,
            "num context": len(context_docs)
        })
    
    # 結果をDataFrameに変換
    eval_df = pd.DataFrame(p_data)
    
    return eval_df

In [104]:
# 評価用データ作成実行
eval_df = evaluate_rag_chain(
    rag_chain=rewrite_retrieve_read_chain,
    retriever=retriever,
    file_path=file_path
)

# 評価結果の確認
print(f"評価対象サンプル数: {len(eval_df)}")
eval_df.head()

評価対象サンプル数: 7


,question,target_answer,response,context,num context
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,オープンイノベーション促進税制におけるスタートアップ企業の株式取得に対する税制優遇措置は、新...,[page_content=' 国内事業会社又はその国内CVCが、オープンイノベーションに...,4
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,控除対象となる研究開発活動は、企業が主に国内で自ら開発した知的財産に限られます。,[page_content=' 我が国のイノベーション拠点の立地競争力を強化する観点から、...,4
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,「カーブアウト加速等支援事業」の主な目的は、事業会社に蓄積されている技術を活用し、新たな会社...,[page_content=' 日本企業では、研究開発により得た技術であって、事業化されな...,4
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,I don't know.,[page_content=' 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地...,4
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立した２号ファンドの資金規...,[page_content=' 2023年１月、産業革新投資機構（JIC）の子会社であるベ...,4


In [105]:
import pandas as pd
from langchain.evaluation import load_evaluator
from langchain.evaluation import EvaluatorType

relevance_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="relevance",
    model="gpt-4o-mini",
    model_kwargs={"temperature": 0}
)

accuracy_evaluator = load_evaluator(
    EvaluatorType.QA,
    criteria="accuracy",
    model="gpt-4o-mini",
    model_kwargs={"temperature": 0}
)

def evaluate_response(df):
    """
    与えられたデータリストに対する Relevance と Accuracy の評価を行い、結果を DataFrame で返します。
    """
    def _flatten_context(context_val):
        if isinstance(context_val, list):
            return "\n".join([item.page_content if hasattr(item, "page_content") else str(item) for item in context_val])
        return str(context_val)
    
    df["ContextText"] = df["context"].apply(lambda x: _flatten_context(x))

    def _evaluate_row(row):
        relevance_score = relevance_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["ContextText"]
        )
        accuracy_score = accuracy_evaluator.evaluate_strings(
            prediction=row["response"],
            input=row["question"],
            reference=row["target_answer"]
        )
        return relevance_score, accuracy_score
    
    df[["Relevance", "Accuracy"]] = df.apply(lambda row: pd.Series(_evaluate_row(row)), axis=1)
    
    df["RelevanceScore"] = df["Relevance"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    df["AccuracyScore"] = df["Accuracy"].apply(lambda x: 1 if x["value"].upper() == "CORRECT" else 0)
    
    # 平均スコアを計算
    avg_relevance = df["RelevanceScore"].mean()
    avg_accuracy = df["AccuracyScore"].mean()
    
    print(f"Average Relevance: {avg_relevance:.2f}")
    print(f"Average Accuracy: {avg_accuracy:.2f}")

    # きれいに処理したContextTextカラムがあるので、その前段階のcontextカラムを削除して返す
    return df.drop(columns=['context'])

In [106]:
results = evaluate_response(eval_df)
results 

Average Relevance: 0.57
Average Accuracy: 0.57


,question,target_answer,response,num context,ContextText,Relevance,Accuracy,RelevanceScore,AccuracyScore
0,オープンイノベーション促進税制において、スタートアップ企業の株式取得に対する税制優遇措置は、...,オープンイノベーション促進税制の下で、新規発行株式の取得は「新規出資型」として分類され、発行...,オープンイノベーション促進税制におけるスタートアップ企業の株式取得に対する税制優遇措置は、新...,4, 国内事業会社又はその国内CVCが、オープンイノベーションにより新事業開拓・生産性向上を図...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
1,イノベーション拠点税制における所得控除について、控除対象となる研究開発活動に関して具体的にど...,イノベーション拠点税制における所得控除の対象となるためには、企業が主に「国内で」「自ら」開発...,控除対象となる研究開発活動は、企業が主に国内で自ら開発した知的財産に限られます。,4, 我が国のイノベーション拠点の立地競争力を強化する観点から、海外と比べて遜色ない事業環境の...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
2,「カーブアウト加速等支援事業」の主な目的は何ですか？,事業会社に蓄積されている技術を活用し、新たな会社を立ち上げた者や立ち上げる意思を持つ者に研究...,「カーブアウト加速等支援事業」の主な目的は、事業会社に蓄積されている技術を活用し、新たな会社...,4, 日本企業では、研究開発により得た技術であって、事業化されないものの多くが消滅しており、研...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
3,グローバル・アクセラレーション・ハブの拠点がある北米の都市を全て教えてください。,グローバル・アクセラレーション・ハブの北米の拠点は、ボストン、ニューヨーク、シカゴ、オーステ...,I don't know.,4, 独立行政法人日本貿易振興機構（JETRO）\nは、世界８地域・３０都市において、現地\n...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
4,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立したファンドについて、資...,産業革新投資機構(JIC)は子会社であるベンチャー・グロース・インベストメンツ(VGI)を通...,産業革新投資機構がベンチャー・グロース・インベストメンツを通じて設立した２号ファンドの資金規...,4, 2023年１月、産業革新投資機構（JIC）の子会社であるベンチャー・グロース・インベスト...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1
5,スタートアップ支援資金と挑戦支援資本強化特別貸付の融資限度額と返済期間の違いに加えて、要件や...,スタートアップ支援資金は融資限度額が20億円で直接貸付、返済期間は20年以内、要件はJVCA...,I don't know.,4, スタートアップの創業等を促進するため、日本政策金融公庫等において、創業等関連の融資・保証...,"{'reasoning': 'INCORRECT', 'value': 'INCORRECT...","{'reasoning': 'INCORRECT', 'value': 'INCORRECT...",0,0
6,宇宙戦略基金の設立と関連する技術開発テーマの具体的な支援分野について説明し、各分野間でどのよ...,宇宙戦略基金は、民間企業や大学、スタートアップ、国立研究機関に対して10年間にわたる研究開発...,宇宙戦略基金は、我が国として推進すべき技術開発テーマを設定し、そのテーマに基づいて支援を行っ...,4,「宇宙戦略基金」\n■ 概要\n■ スキーム・要件\n■ 実績・アピールポイント\n予算額\...,"{'reasoning': 'CORRECT', 'value': 'CORRECT', '...","{'reasoning': 'CORRECT', 'value': 'CORRECT', '...",1,1


![langsmithの画面](https://i.imgur.com/VFTF4JI.png)